1. Import libs and read excel (reading excel is slow). Remember to change the filename to the correct excel.

In [12]:
from pprint import pprint
import pandas as pd
import re

excel_filepath = '../../Clarifications/G3 - mandatory data elements 11.10.2023 (2).xlsx'
data = pd.read_excel(excel_filepath)
print("Finished loading excel \"" + excel_filepath + "\".")


Finished loading excel "../../Clarifications/G3 - mandatory data elements 11.10.2023 (2).xlsx".


2. Extract elements and sub-element names from clarifications excel sheet. Remember to change 'declaration_type' below to the correct value.

In [13]:
'''Remember to change this to your own declaration type.'''
declaration_type = 'G3'

clarification_elements = []
clarification_elements_w_card = {}

#print(data.keys())

for i in range(data.shape[0]):
    element = data.at[i, 'Data element/class name ']
    card = data.at[i,'Should the data element (from column B) be filled out on the declaration? ']
    if element != element:
        element = data.at[i, 'Data sub-element/sub-class name']
        card = data.at[i,'If the data element (from column B) is being declared on the declaration - which data sub-elements (from column C) should then ALWAYS also be filled out? ']
    if card == 'M':
        cnt = 0
        if 'MC' in data.at[i, declaration_type]:
            cnt +=1
        if 'MI' in data.at[i, declaration_type]:
            cnt +=1
        if 'HC' in data.at[i, declaration_type]:
            cnt +=1
        if 'HI' in data.at[i, declaration_type]:
            cnt +=1
        if cnt > 1:
            card_1 = '0'
        else: 
            card_1 = '1'
    else:
            card_1 = '0'
    clarification_elements.append(data.at[i, 'DE No. (Ref ID6)'] + " " + element.lower())
    clarification_elements_w_card[data.at[i, 'DE No. (Ref ID6)'] + " " + element.lower()] = card_1
print(clarification_elements)
print(clarification_elements_w_card)
print("Finished reading elements from clarifications excel sheet.")


['12 01 000 000 previous document', '12 01 001 000 reference number', '12 01 002 000 type', '12 01 007 000 goods item identifier', '12 05 000 000 transport document', '12 05 001 000 reference number', '12 05 002 000 type', '12 09 000 000 lrn', '12 09 001 000 lrn', '13 05 000 000 declarant', '13 05 016 000 name', '13 05 017 000 identification number', '13 05 018 000 address', '13 05 018 023 street', '13 05 018 024 street additional line', '13 05 018 025 number', '13 05 018 026 p.o. box', '13 05 018 027 sub-division', '13 05 018 020 country', '13 05 018 021 postcode', '13 05 018 022 city', '13 05 029 000 communication', '13 05 029 015 identifier', '13 05 029 002 type', '13 06 000 000 representative', '13 06 016 000 name', '13 06 017 000 identification number', '13 06 030 000 status', '13 06 029 000 communication', '13 06 029 015 identifier', '13 06 029 002 type', '16 15 000 000 location of goods', '16 15 045 000 type of location', '16 15 046 000 qualifier of identification', '16 15 036 0

3. Extract EUCDM name annotations from XSD file

(NOTE: this assumes all elements are correctly annotated, which for G4 they are because I manually edited them, ALWAYS annotate the elements right under the element header!!!!)

Remember to change the XSD filename below.

In [14]:
'''Change this filename to the XSD you want to edit.'''
XSD_filename = './G4_DMS_v0.1.xsd'

def extract_elements(text, search_by):
    # Define the regex pattern
    if search_by == 'Name':
        pattern = r"<EUCDMname>(.*?)</EUCDMname>"
    elif search_by == 'Number':
        pattern = r"<code>(.*?)</code>"
    elif search_by == 'Both':
        pattern = (
                r"<EUCDMname>(.*?)</EUCDMname>\s*"
                r"<EUCDMversion>(.*?)</EUCDMversion>\s*"
                r"<code>(.*?)</code>"
            )
    elif search_by == 'get element name':
        pattern = (
                r"<xs:element name=(.*?)\s*"
                r"<xs:annotation>\s*"
                r"<xs:documentation>\s*"
                r"<EUCDMname>(.*?)</EUCDMname>\s*"
                r"<EUCDMversion>(.*?)</EUCDMversion>\s*"
                r"<code>(.*?)</code>"
            )


    # Use re.findall to extract all occurrences
    matches = re.findall(pattern, text)
    # Return the matches
    return matches

text = open(XSD_filename, 'r').read()
# element_name_annotations = extract_elements(text, 'Name')
# element_number_annotations = extract_elements(text, 'Number')
annotations = extract_elements(text, 'Both')

elements = extract_elements(text, 'get element name')
element_names = []
for i in range(len(elements)):
    element_names.append(elements[i][0])

schema_annotations = []
for i in range(len(annotations)):
    schema_annotations.append(annotations[i][2] + " " + annotations[i][0].lower()) 

print(schema_annotations)

optionality = [] # 0 = optional, 1 = mandatory
for element in element_names:
    if "minOccurs=\"0\"" in element:
        optionality.append(0)
    else:
        optionality.append(1)

schema_annotations_w_card = {}
for i in range(len(element_names)):
    schema_annotations_w_card[elements[i][3] + " " + elements[i][1].lower()] = optionality[i]
print(schema_annotations_w_card)
print("Finished extracting XSD elements.")





['12 09 001 000 lrn', '11 04 001 000 specific circumstance indicator', '13 06 000 000 representative', '13 06 016 000 name', '13 06 017 000 identification number', '13 06 018 000 address', '13 06 018 022 city', '13 06 018 020 country', '13 06 018 024 street additional line', '13 06 018 021 postcode', '13 06 018 023 street', '13 06 018 025 number', '13 06 018 026 p.o. box', '13 06 018 027 sub-division', '13 06 029 000 communication', '13 06 029 015 identifier', '13 06 029 002 type', '13 06 030 000 status', '13 05 000 000 declarant', '13 05 016 000 name', '13 05 017 000 identification number', '13 05 018 000 address', '13 05 018 022 city', '13 05 018 020 country', '13 05 018 024 street additional line', '13 05 018 021 postcode', '13 05 018 023 street', '13 05 018 025 number', '13 05 018 026 p.o. box', '13 05 018 027 sub-division', '13 05 029 000 communication', '13 05 029 015 identifier', '13 05 029 002 type', '15 04 001 000 estimated date and time of arrival at the port of unloading', '

4. Check if elements are missing from XSD or if elements need to be removed from XSD.

In [15]:
#schema_annotations =  {k.lower(): v for k, v in schema_annotations.items()}
print("The following elements from the clarifications excel sheet are missing from the XSD: ")
for item in clarification_elements:
    if item not in schema_annotations: #or val not in schema_annotations.values():
        print(item)
print("\n")

print("The following elements in the XSD are not present in the clarifications excel sheet: ")
for item in schema_annotations:
    if item not in clarification_elements: #or val not in schema_annotations.values():
        print(item)


The following elements from the clarifications excel sheet are missing from the XSD: 
12 09 000 000 lrn
16 15 048 000 gnss
16 15 081 000 postcode address
16 15 081 021 postcode
19 11 000 000 receptacle identification number


The following elements in the XSD are not present in the clarifications excel sheet: 
11 04 001 000 specific circumstance indicator
13 06 018 000 address
13 06 018 022 city
13 06 018 020 country
13 06 018 024 street additional line
13 06 018 021 postcode
13 06 018 023 street
13 06 018 025 number
13 06 018 026 p.o. box
13 06 018 027 sub-division
15 04 001 000 estimated date and time of arrival at the port of unloading
17 10 000 000 supervising customs office
17 10 001 000 reference number
18 03 001 000 total gross mass
15 04 001 000 estimated date and time of arrival at the port of unloading
12 02 000 000 additional information
12 02 008 000 code
12 02 009 000 text
12 04 000 000 additional reference
12 04 002 000 type
13 14 000 000 additional supply chain actor
13 

5. Check that optionality is correct: are there any elements that are optional that should be mandatory and vice versa?

In [10]:

for key, value in clarification_elements_w_card.items():
    try:
        if int(schema_annotations_w_card[key]) != int(value):
            #print(key)
            #print(schema_annotations_w_card[key], value)
            map_dict = {'1' : 'mandatory', '0' : 'optional'}
            print(key + ": XSD optionality " + map_dict[str(schema_annotations_w_card[key])] + " but should be " + map_dict[value])
    except(KeyError):
        pass

11 03 001 000 goods item number: XSD optionality mandatory but should be optional
11 04 001 000 specific circumstance indicator: XSD optionality optional but should be mandatory
12 01 001 000 reference number: XSD optionality mandatory but should be optional
12 01 002 000 type: XSD optionality mandatory but should be optional
12 01 007 000 goods item identifier: XSD optionality mandatory but should be optional
12 02 008 000 code: XSD optionality mandatory but should be optional
12 03 001 000 reference number: XSD optionality mandatory but should be optional
12 03 002 000 type: XSD optionality mandatory but should be optional
12 04 002 000 type: XSD optionality mandatory but should be optional
12 05 000 000 transport document: XSD optionality mandatory but should be optional
12 05 001 000 reference number: XSD optionality mandatory but should be optional
12 05 002 000 type: XSD optionality mandatory but should be optional
12 08 001 000 reference number/ucr: XSD optionality mandatory but